---
# Transformer XL
---

---
# Import Libraries

In [3]:
import os
import sys

In [4]:
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

In [5]:
sys.path.append("../../deep-learning-dna")
sys.path.append("../../settransformer")

In [6]:
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import display
import math
import string

import settransformer as stf
from common.models import dnabert
from common import dna
from lmdbm import Lmdb
from common.data import DnaSequenceGenerator, DnaLabelType, DnaSampleGenerator, find_dbs
import wandb

import tf_utils as tfu

In [7]:
strategy = tfu.devices.select_gpu(0, use_dynamic_memory=True)

---
# Load Data

In [8]:
#Import pretrained model
api = wandb.Api()
model_path = api.artifact("sirdavidludwig/deep-learning-dna/dnabert-pretrain-ablation-dim:8dim").download()
pretrained_model = dnabert.DnaBertModel.load(model_path)
pretrained_model

In [9]:
#Load datafiles
dataset_path = api.artifact("sirdavidludwig/nachusa-dna/dnasamples:v1").download('/data/dna_samples:v1')
samples = find_dbs(dataset_path + '/train')
samples[13]

wandb: Downloading large artifact dnasamples:v1, 4086.55MB. 1260 files... Done. 0:0:0.2


'/data/dna_samples:v1/train/WS-CCW-Jul2015_S82_L001_R1_001.db'

---
# Create Dataset

In [10]:
set_len = 400
sequence_len = 150
kmer = 3
batch_size = 20
batches_per_epoch = 20
augument = True
labels = DnaLabelType.SampleIds
seed = 0
rng = np.random.default_rng(seed)
random_samples = samples.copy()

In [11]:
dataset = DnaSampleGenerator(samples=samples[0:5], subsample_length = set_len, sequence_length=sequence_len,kmer=kmer,batch_size=batch_size,batches_per_epoch=batches_per_epoch,augment=augument,labels=labels, rng=rng)

In [12]:
rng.shuffle(random_samples)

In [13]:
random_samples[0:5]

['/data/dna_samples:v1/train/WS-MU-Oct2016_S71_L001_R1_001.db',
 '/data/dna_samples:v1/train/WS-HPN-Jul2016_S78_L001_R1_001.db',
 '/data/dna_samples:v1/train/Wes21-8-TCR_S22_L001_R1_001.db',
 '/data/dna_samples:v1/train/WS-CCE-Sep2015_S60_L001_R1_001.db',
 '/data/dna_samples:v1/train/Wesley056-NegCtrl_S195_L001_R1_001.db']

--- 
# Batch Parameters

In [14]:
block_size = 200
seq_len = set_len
maxlen = set_len
vocab_size = 5
num_chars_data = set_len*sequence_len
max_files = len(dataset.samples)
max_files

5

In [15]:
if block_size-2 > seq_len:
    raise ValueError("Block size should not be bigger than sequence length")

In [16]:
print(maxlen)
print(vocab_size)
print(num_chars_data)

400
5
60000


---
# Create Embeddings

In [17]:
#Create 8 dimensional embeddings
pretrained_encoder = dnabert.DnaBertEncoderModel(pretrained_model.base)
pretrained_encoder.trainable = False

In [18]:
pretrained_encoder(dataset[0][0][0])

<tf.Tensor: shape=(400, 8), dtype=float32, numpy=
array([[ -7.4475875 ,   1.1663623 ,   6.6939783 , ...,  -7.992336  ,
          1.3616216 ,  -1.1660289 ],
       [  7.202399  ,   2.545476  ,   5.7800546 , ...,  -0.35330856,
          6.0580387 ,   1.5731785 ],
       [ -7.3592477 ,   4.8254547 ,  -6.1533613 , ..., -12.36576   ,
          5.8514266 ,  -5.187319  ],
       ...,
       [ -9.9023075 ,  -1.4885719 ,   0.3919065 , ...,  -8.553962  ,
         -4.919237  ,   4.0787582 ],
       [ -0.14813566,   1.0515904 ,   7.329042  , ...,   3.18255   ,
          4.312391  ,   3.7256906 ],
       [-12.081191  ,  -1.2224596 ,   2.1566005 , ..., -13.340243  ,
         -7.8641644 ,   2.6917217 ]], dtype=float32)>

In [19]:
class Create_Embeddings(keras.layers.Layer):
    def __init__(self, encoder):
        super(Create_Embeddings, self).__init__()
        self.encoder = encoder
        
    def subbatch_predict(self, model, batch, subbatch_size, concat=lambda old, new: tf.concat((old, new), axis=0)):
        def predict(i, result=None):
            n = i + subbatch_size
            pred = tf.stop_gradient(model(batch[i:n]))
            if result is None:
                return [n, pred]
            return [n, concat(result, pred)]
        i, result = predict(0)
        batch_size = tf.shape(batch)[0]
        i, result = tf.while_loop(
            cond=lambda i, _: i < batch_size,
            body=predict,
            loop_vars=[i, result],
            parallel_iterations=1)

        return result
    
    def modify_data_for_input(self, data):
        batch_size = tf.shape(data)[0]
        subsample_size = tf.shape(data)[1]
        flat_data = tf.reshape(data, (batch_size*subsample_size, -1))
        encoded = self.subbatch_predict(self.encoder, flat_data, subsample_size)
        return tf.reshape(encoded, (batch_size, subsample_size, -1))
    
    def call(self, data):
        return  self.modify_data_for_input(data)

---
# Setup

In [20]:
def clone_initializer(initializer):
    if isinstance(initializer, tf.keras.initializers.Initializer):
        return initializer.__class__.from_config(initializer.get_config())
    return initializer

In [21]:
def get_shape_list(tensor, expected_rank=None, name=None):
    if expected_rank is not None:
        assert_rank(tensor, expected_rank, name)

    shape = tensor.shape.as_list()

    non_static_indexes = []
    for (index, dim) in enumerate(shape):
        if dim is None:
            non_static_indexes.append(index)

    if not non_static_indexes:
        return shape

    dyn_shape = tf.shape(tensor)
    for index in non_static_indexes:
        shape[index] = dyn_shape[index]
    return shape

---
# Masked Token and Position Embedding

In [22]:
class MaskedTokenAndPositionEmbedding(keras.layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim, **kwargs):
        super(MaskedTokenAndPositionEmbedding, self).__init__(**kwargs)
        self.token_emb = keras.layers.Embedding(input_dim=vocab_size,
        output_dim=embed_dim,
        mask_zero=True)
        self.pos_emb = keras.layers.Embedding(input_dim=maxlen+1,
        output_dim=embed_dim,
        mask_zero=True)
    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=1, limit=maxlen+1, delta=1)
        positions = positions * tf.cast(tf.sign(x),tf.int32)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions

---
# Custom Loss and Accuracy

In [23]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')
def MaskedSparseCategoricalCrossentropy(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)
    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask
    return tf.reduce_sum(loss_)/tf.reduce_sum(mask)
def MaskedSparseCategoricalAccuracy(real, pred):
    accuracies = tf.equal(tf.cast(real,tf.int64), tf.argmax(pred, axis=2))
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    accuracies = tf.math.logical_and(mask, accuracies)
    accuracies = tf.cast(accuracies, dtype=tf.float32)
    mask = tf.cast(mask, dtype=tf.float32)
    return tf.reduce_sum(accuracies)/tf.reduce_sum(mask)

---
# Cache Memory

In [24]:
# current_state: `Tensor`, the current state.
# previous_state: `Tensor`, the previous state.
# memory_length: `int`, the number of tokens to cache.
# reuse_length: `int`, the number of tokens in the current batch to be cached
#     and reused in the future.
# Returns:  `Tensor`, representing the cached state with stopped gradients.
def _cache_memory(current_state, previous_state, memory_length, reuse_length=0):
    if memory_length is None or memory_length == 0:
        return None
    else:
        if reuse_length > 0:
            current_state = current_state[:, :reuse_length, :]

        if previous_state is None:
            new_mem = current_state[:, -memory_length:, :]
        else:
            new_mem = tf.concat(
                    [previous_state, current_state], 1)[:, -memory_length:, :]

    return tf.stop_gradient(new_mem)

---
# MultiHead Attention

In [25]:
# query: Query `Tensor` of shape `[B, T, dim]`.
# value: Value `Tensor` of shape `[B, S, dim]`.
# content_attention_bias: Bias `Tensor` for content based attention of shape
# `[num_heads, dim]`.
# positional_attention_bias: Bias `Tensor` for position based attention of
# shape `[num_heads, dim]`.
# key: Optional key `Tensor` of shape `[B, S, dim]`. If not given, will use
# `value` for both `key` and `value`, which is the most common case.
# relative_position_encoding: Relative positional encoding `Tensor` of shape
# `[B, L, dim]`.
# state: Optional `Tensor` of shape `[B, M, E]` where M is the length of the
# state or memory. If passed, this is also attended over as in Transformer
# XL.
# attention_mask: A boolean mask of shape `[B, T, S]` that prevents attention
# to certain positions.
class MultiHeadRelativeAttention(tf.keras.layers.MultiHeadAttention):
    def __init__(self,
                 kernel_initializer="variance_scaling",
                 **kwargs):
        super().__init__(kernel_initializer=kernel_initializer,
                                         **kwargs)

    def _build_from_signature(self, query, value, key=None):
        super(MultiHeadRelativeAttention, self)._build_from_signature(
                query=query,
                value=value,
                key=key)
        if hasattr(value, "shape"):
            value_shape = tf.TensorShape(value.shape)
        else:
            value_shape = value
        if key is None:
            key_shape = value_shape
        elif hasattr(key, "shape"):
            key_shape = tf.TensorShape(key.shape)
        else:
            key_shape = key

        common_kwargs = dict(
                kernel_initializer=self._kernel_initializer,
                bias_initializer=self._bias_initializer,
                kernel_regularizer=self._kernel_regularizer,
                bias_regularizer=self._bias_regularizer,
                activity_regularizer=self._activity_regularizer,
                kernel_constraint=self._kernel_constraint,
                bias_constraint=self._bias_constraint)

        with tf.init_scope():
            einsum_equation, _, output_rank = _build_proj_equation(
                    key_shape.rank - 1, bound_dims=1, output_dims=2)
            self._encoding_dense = tf.keras.layers.experimental.EinsumDense(
                    einsum_equation,
                    output_shape=_get_output_shape(
                        output_rank - 1,
                        [self._num_heads, self._key_dim]),
                        bias_axes=None,
                        name="encoding",
                        **common_kwargs)

# query: Projected query `Tensor` of shape `[B, T, N, key_dim]`.
# key: Projected key `Tensor` of shape `[B, S + M, N, key_dim]`.
# value: Projected value `Tensor` of shape `[B, S + M, N, key_dim]`.
# position: Projected position `Tensor` of shape `[B, L, N, key_dim]`.
# content_attention_bias: Trainable bias parameter added to the query head
#     when calculating the content-based attention score.
# positional_attention_bias: Trainable bias parameter added to the query
#     head when calculating the position-based attention score.
# attention_mask: (default None) Optional mask that is added to attention
#     logits. If state is not None, the mask source sequence dimension should
#     extend M.
# Returns:
# attention_output: Multi-headed output of attention computation of shape
#     `[B, S, N, key_dim]`.
    def compute_attention(
        self,
        query,
        key,
        value,
        position,
        content_attention_bias,
        positional_attention_bias,
        attention_mask=None
    ):
        
        #AC
        content_attention = tf.einsum(self._dot_product_equation, key, query + content_attention_bias)
        
        #positional_attention = tf.einsum(self._dot_product_equation, position, query + positional_attention_bias)
        
        
        #BD
        #positional_attention = _rel_shift(positional_attention, klen=tf.shape(content_attention)[3])
        
        attention_sum = content_attention# + positional_attention

        attention_scores = tf.multiply(attention_sum, 1.0 / math.sqrt(float(self._key_dim)))

        attention_scores = self._masked_softmax(attention_scores, attention_mask)

        attention_output = self._dropout_layer(attention_scores)

        attention_output = tf.einsum(self._combine_equation,
                                                                 attention_output,
                                                                 value)
        return attention_output

# * Number of heads (H): the number of attention heads.
# * Value size (V): the size of each value embedding per head.
# * Key size (K): the size of each key embedding per head. Equally, the size
#     of each query embedding per head. Typically K <= V.
# * Batch dimensions (B).
# * Query (target) attention axes shape (T).
# * Value (source) attention axes shape (S), the rank must match the target.
# * Encoding length (L): The relative positional encoding length.
# query: attention input.
# value: attention input.
# content_attention_bias: A trainable bias parameter added to the query head
#     when calculating the content-based attention score.
# positional_attention_bias: A trainable bias parameter added to the query
#     head when calculating the position-based attention score.
# key: attention input.
# relative_position_encoding: relative positional encoding for key and
#     value.
# state: (default None) optional state. If passed, this is also attended
#     over as in TransformerXL.
# attention_mask: (default None) Optional mask that is added to attention
#     logits. If state is not None, the mask source sequence dimension should
#     extend M.
# Returns:
# attention_output: The result of the computation, of shape [B, T, E],
#     where `T` is for target sequence shapes and `E` is the query input last
#     dimension if `output_shape` is `None`. Otherwise, the multi-head outputs
#     are projected to the shape specified by `output_shape`.
    def call(self,
             query,
             value,
             content_attention_bias,
             positional_attention_bias,
             key=None,
             relative_position_encoding=None,
             state=None,
             attention_mask=None):
        
        if not self._built_from_signature:
            self._build_from_signature(query, value, key=key)
        if key is None:
            key = value
        if state is not None and state.shape.ndims > 1:
            value = tf.concat([state, value], 1)
            key = tf.concat([state, key], 1)

        # `query` = [B, T, N ,H]
        query = self._query_dense(query)

        # `key` = [B, S + M, N, H]
        key = self._key_dense(key)

        # `value` = [B, S + M, N, H]
        value = self._value_dense(value)

        # `position` = [B, L, N, H]
        #position = self._encoding_dense(relative_position_encoding)
        position = None
        
        attention_output = self.compute_attention(
                query=query,
                key=key,
                value=value,
                position=position,
                content_attention_bias=content_attention_bias,
                positional_attention_bias=positional_attention_bias,
                attention_mask=attention_mask)

        # `attention_output` = [B, S, N, H]
        attention_output = self._output_dense(attention_output)

        return attention_output

---
# Relative Shift

In [26]:
def _rel_shift(x, klen=-1):    
    x = tf.transpose(x, perm=[2, 3, 0, 1])
    x = tf.pad(x, [[0, 0], [1, 0], [0, 0], [0, 0]])
    x_size = tf.shape(x)
    x = tf.reshape(x, [x_size[1], x_size[0], x_size[2], x_size[3]])
    x = tf.slice(x, [1, 0, 0, 0], [-1, -1, -1, -1])
    x = tf.reshape(x, [x_size[0], x_size[1] - 1, x_size[2], x_size[3]])
    x = tf.transpose(x, perm=[2, 3, 0, 1])

    return x

---
# Build Einsum Equation

In [27]:
_CHR_IDX = string.ascii_lowercase

# Builds an einsum equation for projections inside multi-head attention
def _build_proj_equation(free_dims, bound_dims, output_dims):
    input_str = ""
    kernel_str = ""
    output_str = ""
    bias_axes = ""
    letter_offset = 0
    for i in range(free_dims):
        char = _CHR_IDX[i + letter_offset]
        input_str += char
        output_str += char

    letter_offset += free_dims
    for i in range(bound_dims):
        char = _CHR_IDX[i + letter_offset]
        input_str += char
        kernel_str += char

    letter_offset += bound_dims
    for i in range(output_dims):
        char = _CHR_IDX[i + letter_offset]
        kernel_str += char
        output_str += char
        bias_axes += char
    equation = "%s,%s->%s" % (input_str, kernel_str, output_str)

    return equation, bias_axes, len(output_str)


def _get_output_shape(output_rank, known_last_dims):
    return [None] * (output_rank - len(known_last_dims)) + list(known_last_dims)

---
# Relative Position Embedding

In [28]:
# hidden_size: Size of the hidden layer.
# min_timescale: Minimum scale that will be applied at each position
# max_timescale: Maximum scale that will be applied at each position.
class RelativePositionEmbedding(tf.keras.layers.Layer):

    def __init__(self,
                 hidden_size: int,
                 min_timescale: float = 1.0,
                 max_timescale: float = 1.0e4,
                 **kwargs):
        
        if "dtype" not in kwargs:
            kwargs["dtype"] = "float32"

        super().__init__(**kwargs)
        self._hidden_size = hidden_size
        self._min_timescale = min_timescale
        self._max_timescale = max_timescale

    def get_config(self):
        config = {
                "hidden_size": self._hidden_size,
                "min_timescale": self._min_timescale,
                "max_timescale": self._max_timescale,
        }
        base_config = super(RelativePositionEmbedding, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

# inputs: An tensor whose second dimension will be used as `length`. If
# `None`, the other `length` argument must be specified.
# length: An optional integer specifying the number of positions. If both
# `inputs` and `length` are spcified, `length` must be equal to the second
# dimension of `inputs`.
# Returns:
# A tensor in shape of `(length, hidden_size)`.
    def call(self, inputs, length=None):
        if inputs is None and length is None:
            raise ValueError("If inputs is None, `length` must be set in "
                                             "RelativePositionEmbedding().")
        if inputs is not None:
            input_shape = get_shape_list(inputs)
            if length is not None and length != input_shape[1]:
                raise ValueError(
                        "If inputs is not None, `length` must equal to input_shape[1].")
            length = input_shape[1]
        position = tf.cast(tf.range(length), tf.float32)
        num_timescales = self._hidden_size // 2
        min_timescale, max_timescale = self._min_timescale, self._max_timescale
        log_timescale_increment = (
                math.log(float(max_timescale) / float(min_timescale)) /
                (tf.cast(num_timescales, tf.float32) - 1))
        inv_timescales = min_timescale * tf.exp(
                tf.cast(tf.range(num_timescales), tf.float32) *
                -log_timescale_increment)
        scaled_time = tf.expand_dims(position, 1) * tf.expand_dims(
                inv_timescales, 0)
        position_embeddings = tf.concat(
                [tf.sin(scaled_time), tf.cos(scaled_time)], axis=1)
        position_embeddings = tf.expand_dims(position_embeddings, axis=0)
        return tf.tile(position_embeddings, (tf.shape(inputs)[0], 1, 1))

---
# XL Block

In [29]:
# vocab_size: The size of the token vocabulary.
# hidden_size: The size of the transformer hidden layers.
# num_attention_heads: The number of attention heads.
# head_size: The dimension size of each attention head.
# inner_size: The inner size for the transformer layers.
# dropout_rate: Dropout rate for the output of this layer.
# attention_dropout_rate: Dropout rate on attention probabilities.
# norm_epsilon: Epsilon value to initialize normalization layers.
# inner_activation: The activation to use for the inner
#     FFN layers.
# kernel_initializer: Initializer for dense layer kernels.
# inner_dropout: Dropout probability for the inner dropout
#     layer.
class TransformerXLBlock(tf.keras.layers.Layer):
    def __init__(self,
                 vocab_size,
                 hidden_size,
                 num_attention_heads,
                 head_size,
                 inner_size,
                 dropout_rate,
                 attention_dropout_rate,
                 norm_epsilon=1e-12,
                 inner_activation="relu",
                 kernel_initializer="variance_scaling",
                 inner_dropout=0.0,
                 **kwargs):
        """Initializes TransformerXLBlock layer."""

        super(TransformerXLBlock, self).__init__(**kwargs)
        self._vocab_size = vocab_size
        self._num_heads = num_attention_heads
        self._head_size = head_size
        self._hidden_size = hidden_size
        self._inner_size = inner_size
        self._dropout_rate = dropout_rate
        self._attention_dropout_rate = attention_dropout_rate
        self._inner_activation = inner_activation
        self._norm_epsilon = norm_epsilon
        self._kernel_initializer = kernel_initializer
        self._inner_dropout = inner_dropout
        self._attention_layer_type = Set_Transformer
        
    def build(self, input_shape):
        input_tensor = input_shape[0] if len(input_shape) == 2 else input_shape
        input_tensor_shape = tf.TensorShape(input_tensor)
        if len(input_tensor_shape.as_list()) != 3:
            raise ValueError("TransformerLayer expects a three-dimensional input of "
                                             "shape [batch, sequence, width].")
        batch_size, sequence_length, hidden_size = input_tensor_shape

        if len(input_shape) == 2:
            mask_tensor_shape = tf.TensorShape(input_shape[1])
            expected_mask_tensor_shape = tf.TensorShape(
                    [batch_size, sequence_length, sequence_length])
            if not expected_mask_tensor_shape.is_compatible_with(mask_tensor_shape):
                raise ValueError("When passing a mask tensor to TransformerXLBlock, "
                                                 "the mask tensor must be of shape [batch, "
                                                 "sequence_length, sequence_length] (here %s). Got a "
                                                 "mask tensor of shape %s." %
                                                 (expected_mask_tensor_shape, mask_tensor_shape))
        if hidden_size % self._num_heads != 0:
            raise ValueError(
                    "The input size (%d) is not a multiple of the number of attention "
                    "heads (%d)" % (hidden_size, self._num_heads))
            
            
        self._attention_layer= self._attention_layer_type(embed_dim=self._head_size,num_heads=self._num_heads, stack = 4,
                                                   use_layernorm=True,pre_layernorm=True,use_keras_mha=True)
        
 
        
        # self._attention_layer = self._attention_layer_type(
        #         num_heads=self._num_heads,
        #         key_dim=self._head_size,
        #         value_dim=self._head_size,
        #         dropout=self._attention_dropout_rate,
        #         use_bias=False,
        #         kernel_initializer=clone_initializer(self._kernel_initializer),
        #         name="rel_attn")
        self._attention_dropout = tf.keras.layers.Dropout(
                rate=self._attention_dropout_rate)
        self._attention_layer_norm = tf.keras.layers.LayerNormalization(
                name="self_attention_layer_norm",
                axis=-1,
                epsilon=self._norm_epsilon,
                dtype=tf.float32)
        self._inner_dense = tf.keras.layers.experimental.EinsumDense(
                "abc,cd->abd",
                output_shape=(None, self._inner_size),
                bias_axes="d",
                kernel_initializer=clone_initializer(self._kernel_initializer),
                name="inner")

        self._inner_activation_layer = tf.keras.layers.Activation(
                self._inner_activation)
        self._inner_dropout_layer = tf.keras.layers.Dropout(
                rate=self._inner_dropout)
        self._output_dense = tf.keras.layers.experimental.EinsumDense(
                "abc,cd->abd",
                output_shape=(None, hidden_size),
                bias_axes="d",
                name="output",
                kernel_initializer=clone_initializer(self._kernel_initializer))
        self._output_dropout = tf.keras.layers.Dropout(rate=self._dropout_rate)
        self._output_layer_norm = tf.keras.layers.LayerNormalization(
                name="output_layer_norm",
                axis=-1,
                epsilon=self._norm_epsilon)

        super(TransformerXLBlock, self).build(input_shape)

    def get_config(self):
        config = {
                "vocab_size":
                        self._vocab_size,
                "hidden_size":
                        self._hidden_size,
                "num_attention_heads":
                        self._num_heads,
                "head_size":
                        self._head_size,
                "inner_size":
                        self._inner_size,
                "dropout_rate":
                        self._dropout_rate,
                "attention_dropout_rate":
                        self._attention_dropout_rate,
                "norm_epsilon":
                        self._norm_epsilon,
                "inner_activation":
                        self._inner_activation,
                "kernel_initializer":
                        self._kernel_initializer,
                "inner_dropout":
                        self._inner_dropout,
        }
        base_config = super(TransformerXLBlock, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

# content_stream: `Tensor`, the input content stream. This is the standard
# input to Transformer XL and is commonly referred to as `h` in XLNet.
# content_attention_bias: Bias `Tensor` for content based attention of shape
# `[num_heads, dim]`.
# positional_attention_bias: Bias `Tensor` for position based attention of
# shape `[num_heads, dim]`.
# relative_position_encoding: Relative positional encoding `Tensor` of shape
# `[B, L, dim]`.
# state: Optional `Tensor` of shape `[B, M, E]`, where M is the length of
# the state or memory. If passed, this is also attended over as in
# Transformer XL.
# content_attention_mask: Optional `Tensor` representing the mask that is
# added to content attention logits. If state is not None, the mask source
# sequence dimension should extend M.
# query_attention_mask: Optional `Tensor` representing the mask that is
# added to query attention logits. If state is not None, the mask source
# sequence dimension should extend M.
# target_mapping: Optional `Tensor` representing the target mapping when
# calculating query attention.
# Returns:
# A `dict` object, containing the key value pairs for `content_attention`
    def call(self,
             content_stream,
             content_attention_bias,
             positional_attention_bias,
             relative_position_encoding=None,
             state=None,
             content_attention_mask=None,
             query_attention_mask=None,
             target_mapping=None):
        
        attention_kwargs = dict(
                query=content_stream,
                value=content_stream,
                key=content_stream,
                attention_mask=content_attention_mask)

        common_attention_kwargs = dict(
                content_attention_bias=content_attention_bias,
                relative_position_encoding=relative_position_encoding,
                positional_attention_bias=positional_attention_bias,
                state=state)

        attention_kwargs.update(common_attention_kwargs)
        #attention_output = self._attention_layer(**attention_kwargs)
        attention_output = {"content_attention": self._attention_layer(content_stream)}
        
#         attention_stream = attention_output
#         input_stream = content_stream
#         attention_key = "content_attention"
#        attention_output = {}
        
#         attention_stream = self._attention_dropout(attention_stream)
#         attention_stream = self._attention_layer_norm(attention_stream + input_stream)
#         inner_output = self._inner_dense(attention_stream)
#         inner_output = self._inner_activation_layer(
#                 inner_output)
#         inner_output = self._inner_dropout_layer(
#                 inner_output)
#         layer_output = self._output_dense(inner_output)
#         layer_output = self._output_dropout(layer_output)
#         layer_output = self._output_layer_norm(layer_output + attention_stream)
#         attention_output[attention_key] = layer_output

        return attention_output

---
# Transformer XL

In [30]:
# num_layers: The number of layers.
# hidden_size: The hidden size.
# num_attention_heads: The number of attention heads.
# head_size: The dimension size of each attention head.
# inner_size: The hidden size in feed-forward layers.
# dropout_rate: Dropout rate used in each Transformer XL block.
# attention_dropout_rate: Dropout rate on attention probabilities.
# initializer: The initializer to use for attention biases.
# tie_attention_biases: Whether or not to tie biases together. If `True`, then
# each Transformer XL block shares the same trainable attention bias. If
# `False`, then each block has its own attention bias. This is usually set
# to `True`.
# memory_length: The number of tokens to cache.
# reuse_length: The number of tokens in the current batch to be cached
# and reused in the future.
# inner_activation: The activation to use in the inner layers
# for Transformer XL blocks. Typically "relu" or "gelu".
class TransformerXL(tf.keras.layers.Layer):
    def __init__(self,
                 vocab_size,
                 num_layers,
                 hidden_size,
                 maxlen,
                 embed_dim,
                 num_attention_heads,
                 head_size,
                 inner_size,
                 dropout_rate,
                 attention_dropout_rate,
                 initializer,
                 tie_attention_biases=True,
                 memory_length=None,
                 reuse_length=None,
                 inner_activation="relu",
                 **kwargs):
        super(TransformerXL, self).__init__(**kwargs)

        self._vocab_size = vocab_size
        self._initializer = initializer
        self._num_layers = num_layers
        self._hidden_size = hidden_size
        self._num_attention_heads = num_attention_heads
        self._head_size = head_size
        self._inner_size = inner_size
        self._inner_activation = inner_activation
        self._dropout_rate = dropout_rate
        self._attention_dropout_rate = attention_dropout_rate
        self._tie_attention_biases = tie_attention_biases

        self._memory_length = memory_length
        self._reuse_length = reuse_length

        if self._tie_attention_biases:
            attention_bias_shape = [self._num_attention_heads, self._head_size]
        else:
            attention_bias_shape = [self._num_layers, self._num_attention_heads, self._head_size]

        self.content_attention_bias = self.add_weight(
                "content_attention_bias",
                shape=attention_bias_shape,
                dtype=tf.float32,
                initializer=clone_initializer(self._initializer))
        self.positional_attention_bias = self.add_weight(
                "positional_attention_bias",
                shape=attention_bias_shape,
                dtype=tf.float32,
                initializer=clone_initializer(self._initializer))

        self.transformer_xl_layers = []
        for i in range(self._num_layers):
            self.transformer_xl_layers.append(
                    TransformerXLBlock(
                            vocab_size=self._vocab_size,
                            hidden_size=self._head_size * self._num_attention_heads,
                            num_attention_heads=self._num_attention_heads,
                            head_size=self._head_size,
                            inner_size=self._inner_size,
                            dropout_rate=self._dropout_rate,
                            attention_dropout_rate=self._attention_dropout_rate,
                            norm_epsilon=1e-12,
                            inner_activation=self._inner_activation,
                            kernel_initializer="variance_scaling",
                            name="layer_%d" % i))

        self.output_dropout = tf.keras.layers.Dropout(rate=self._dropout_rate)

    def get_config(self):
        config = {
                "vocab_size":
                        self._vocab_size,
                "num_layers":
                        self._num_layers,
                "hidden_size":
                        self._hidden_size,
                "num_attention_heads":
                        self._num_attention_heads,
                "head_size":
                        self._head_size,
                "inner_size":
                        self._inner_size,
                "dropout_rate":
                        self._dropout_rate,
                "attention_dropout_rate":
                        self._attention_dropout_rate,
                "initializer":
                        self._initializer,
                "tie_attention_biases":
                        self._tie_attention_biases,
                "memory_length":
                        self._memory_length,
                "reuse_length":
                        self._reuse_length,
                "inner_activation":
                        self._inner_activation,
        }
        base_config = super(TransformerXL, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

# content_stream: `Tensor`, the input content stream. This is the standard
# input to Transformer XL and is commonly referred to as `h` in XLNet.
# relative_position_encoding: Relative positional encoding `Tensor` of shape
# `[B, L, dim]`.
# state: Optional `Tensor` of shape `[B, M, E]`, where M is the length of
# the state or memory. If passed, this is also attended over as in
# Transformer XL.
# content_attention_mask: Optional `Tensor` representing the mask that is
# added to content attention logits. If state is not None, the mask source
# sequence dimension should extend M.
# query_attention_mask: Optional `Tensor` representing the mask that is
# added to query attention logits. If state is not None, the mask source
# sequence dimension should extend M.
# target_mapping: Optional `Tensor` representing the target mapping when
# calculating query attention.
# Returns:
# A tuple consisting of the attention output and the list of cached memory
# states.
# The attention output is `content_attention`
    def call(self,
             content_stream,
             relative_position_encoding,
             state=None,
             content_attention_mask=None,
             query_attention_mask=None,
             target_mapping=None):
        
        new_mems = []

        if state is None:
            state = [None] * self._num_layers
        for i in range(self._num_layers):
            # cache new mems
            new_mems.append(
                    _cache_memory(content_stream, state[i],
                                                self._memory_length, self._reuse_length))

            if self._tie_attention_biases:
                content_attention_bias = self.content_attention_bias
            else:
                content_attention_bias = self.content_attention_bias[i]
                
            if self._tie_attention_biases:
                positional_attention_bias = self.positional_attention_bias
            else:
                positional_attention_bias = self.positional_attention_bias[i]

            transformer_xl_layer = self.transformer_xl_layers[i]
            
            transformer_xl_output = transformer_xl_layer(
                    content_stream=content_stream,
                    content_attention_bias=content_attention_bias,
                    positional_attention_bias=positional_attention_bias,
                    relative_position_encoding=relative_position_encoding,
                    state=state[i],
                    content_attention_mask=content_attention_mask,
                    query_attention_mask=query_attention_mask,
                    target_mapping=target_mapping)
            content_stream = transformer_xl_output["content_attention"]

        output_stream = content_stream
        return output_stream, new_mems

In [39]:
class Set_Transformer(keras.Model):
    def __init__(self, embed_dim, num_heads, stack, use_layernorm, pre_layernorm, use_keras_mha):
        super(Set_Transformer, self).__init__()
        
        self.isabs = []
        
        for i in range(stack):
            self.isabs.append(stf.InducedSetAttentionBlock(embed_dim=embed_dim,num_heads=num_heads,num_induce=24,use_layernorm=use_layernorm,pre_layernorm=pre_layernorm,use_keras_mha=use_keras_mha))
      
    def call(self, data):
            
            attention = data
            
            for isab in self.isabs:
                attention = isab(attention)
                
            return attention

---
# Xl Model Class

In [40]:
class XlModel(keras.Model):
    def __init__(self, max_files, encoder, block_size, seq_len_padded, embed_dim, vocab_size, num_layers, hidden_size, num_attention_heads, maxlen, memory_length, reuse_length, head_size, inner_size, dropout_rate, attention_dropout_rate, initializer):
        super(XlModel, self).__init__()
        
        self.block_size = block_size
        self.seq_len_padded = seq_len_padded
        self.embed_dim = embed_dim
        self.num_heads = num_attention_heads
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.maxlen = maxlen
        self.memory_length = memory_length
        self.max_files = max_files
        self.encoder = encoder
        self.isabs = []
        
        self.embedding_layer = Create_Embeddings(encoder)

        self.linear_layer = keras.layers.Dense(embed_dim)
        
        self.transformer_xl = TransformerXL(
                vocab_size=vocab_size,
                num_layers=num_layers,
                hidden_size=hidden_size,
                num_attention_heads=num_attention_heads,
                maxlen=maxlen,
                embed_dim=embed_dim,
                memory_length=memory_length,
                reuse_length=reuse_length,
                head_size=head_size,
                inner_size=inner_size,
                dropout_rate=dropout_rate,
                attention_dropout_rate=attention_dropout_rate,
                initializer=initializer, 
            )
        

        self.pooling_layer = stf.PoolingByMultiHeadAttention(num_seeds=1,embed_dim=embed_dim,num_heads=1,use_layernorm=True,pre_layernorm=True, use_keras_mha=True,is_final_block=True)
    
        self.reshape_layer = keras.layers.Reshape((embed_dim,))
   
        self.output_layer = keras.layers.Dense(self.max_files, activation=keras.activations.softmax)
        
    
    def call(self, x, training=None):        
 
        mems = tf.zeros((self.num_layers, tf.shape(x)[0], self.memory_length, self.embed_dim))
        
        embeddings = self.embedding_layer(x)
            
        linear_transform = self.linear_layer(embeddings)    
            
        for i in range(0, self.seq_len_padded, self.block_size):
            block = embeddings[:,i:i+block_size]
            
            output, mems = self.transformer_xl(content_stream=block, relative_position_encoding=None, state=mems)
                
        pooling = self.pooling_layer(output)

        reshape = self.reshape_layer(pooling)

        output = self.output_layer(reshape)          
        
        return output

---
# Xl Parameters

In [41]:
#Xl Parameters 
embed_dim = 8
num_layers = 10
hidden_size = 32
num_attention_heads = 8
memory_length = 200
reuse_length = 0
head_size = 8
inner_size = 32
dropout_rate = 0.01
attention_dropout_rate = 0.01
initializer = keras.initializers.RandomNormal(stddev=0.1) 

encoder = pretrained_encoder
epochs = 100

---
# Create Models

In [42]:
model = XlModel(max_files, encoder, block_size, seq_len, embed_dim, vocab_size, num_layers, hidden_size, num_attention_heads, maxlen, memory_length, reuse_length, head_size, inner_size, dropout_rate, attention_dropout_rate, initializer)
model.compile(loss = keras.losses.SparseCategoricalCrossentropy(from_logits=False), optimizer = keras.optimizers.Nadam(1e-4), metrics = keras.metrics.SparseCategoricalAccuracy())
model.fit(dataset, epochs = epochs)

Epoch 1/100
20/20 [==============================] - 174s 3s/step - loss: 1.8640 - sparse_categorical_accuracy: 0.2550
Epoch 2/100
20/20 [==============================] - 64s 3s/step - loss: 1.6861 - sparse_categorical_accuracy: 0.1950
Epoch 3/100
20/20 [==============================] - 63s 3s/step - loss: 1.5989 - sparse_categorical_accuracy: 0.2575
Epoch 4/100
20/20 [==============================] - 64s 3s/step - loss: 1.6171 - sparse_categorical_accuracy: 0.2625
Epoch 5/100
20/20 [==============================] - 63s 3s/step - loss: 1.6002 - sparse_categorical_accuracy: 0.3275
Epoch 6/100
20/20 [==============================] - 64s 3s/step - loss: 1.5489 - sparse_categorical_accuracy: 0.3750
Epoch 7/100
20/20 [==============================] - 64s 3s/step - loss: 1.5522 - sparse_categorical_accuracy: 0.3200
Epoch 8/100
20/20 [==============================] - 64s 3s/step - loss: 1.5219 - sparse_categorical_accuracy: 0.3450
Epoch 9/100
20/20 [==============================] - 64

In [ ]:
u = model.evaluate(dataset[0][0], dataset[0][1])

In [ ]:
u = model.predict(dataset[0][0])

In [ ]:
u = u.argmax(-1)

In [46]:
u

array([0, 4, 2, 4, 0, 2, 4, 4, 0, 4, 0, 1, 1, 4, 0, 2, 2, 1, 2, 2])

In [42]:
dataset[0][1]

array([0, 0, 2, 3, 0, 4, 4, 4, 3, 3, 0, 1, 1, 3, 4, 3, 2, 1, 2, 2],
      dtype=int32)

In [49]:
samples[0:5]

['/data/dna_samples:v1/train/WS-AG-Apr2016_S85_L001_R1_001.db',
 '/data/dna_samples:v1/train/WS-AG-Jul2015_S3_L001_R1_001.db',
 '/data/dna_samples:v1/train/WS-AG-Jul2016_S31_L001_R1_001.db',
 '/data/dna_samples:v1/train/WS-AG-May2015_S65_L001_R1_001.db',
 '/data/dna_samples:v1/train/WS-AG-Oct2016_S72_L001_R1_001.db']